<a href="https://colab.research.google.com/github/Shibu4064/ML_Project/blob/main/classification_with_buetcsenlp_bert_finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets
!pip install transformers
!pip install transformers[torch]
!pip install transformers[tf-cpu]
!pip install transformers[flax]
!pip install torch
!pip install tensorflow
!pip install evaluate
!pip install torchsummary
!pip install gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.6/225.6 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !zip -r /content/drive/MyDrive/EMNLP/fine_tuned_buetcsenlp_bangla_bert1.zip /content/drive/MyDrive/EMNLP/fine_tuned_buetcsenlp_bangla_bert1

updating: content/drive/MyDrive/EMNLP/fine_tuned_buetcsenlp_bangla_bert1/ (stored 0%)
  adding: content/drive/MyDrive/EMNLP/fine_tuned_buetcsenlp_bangla_bert1/config.json (deflated 52%)
  adding: content/drive/MyDrive/EMNLP/fine_tuned_buetcsenlp_bangla_bert1/pytorch_model.bin (deflated 7%)
  adding: content/drive/MyDrive/EMNLP/fine_tuned_buetcsenlp_bangla_bert1/tokenizer_config.json (deflated 47%)
  adding: content/drive/MyDrive/EMNLP/fine_tuned_buetcsenlp_bangla_bert1/special_tokens_map.json (deflated 42%)
  adding: content/drive/MyDrive/EMNLP/fine_tuned_buetcsenlp_bangla_bert1/vocab.txt (deflated 71%)
  adding: content/drive/MyDrive/EMNLP/fine_tuned_buetcsenlp_bangla_bert1/tokenizer.json (deflated 76%)
  adding: content/drive/MyDrive/EMNLP/fine_tuned_buetcsenlp_bangla_bert1/training_args.bin (deflated 49%)


In [2]:
import multiprocessing
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import transformers
import torch.nn as nn
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoModelForMaskedLM, AutoModelForPreTraining, EarlyStoppingCallback, AutoModelForSequenceClassification, BertForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from transformers import BertForMaskedLM, DistilBertForMaskedLM
from transformers import BertTokenizer, DistilBertTokenizer
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from tokenizers import BertWordPieceTokenizer

In [3]:
import gdown

model_link = "https://drive.google.com/uc?id=12S0Pfq7aHhy19S6srv08Pusm1HL5grLJ"
config_link = "https://drive.google.com/uc?id=1-yXyTCRd8uCC9Zca-EigUzuDBqCulP2e"

gdown.download(model_link,"model.bin")
gdown.download(config_link,"config.json")

Downloading...
From: https://drive.google.com/uc?id=12S0Pfq7aHhy19S6srv08Pusm1HL5grLJ
To: /content/model.bin
100%|██████████| 443M/443M [00:06<00:00, 72.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-yXyTCRd8uCC9Zca-EigUzuDBqCulP2e
To: /content/config.json
100%|██████████| 782/782 [00:00<00:00, 2.98MB/s]


'config.json'

In [4]:
from transformers import AutoTokenizer, AutoModel, AutoConfig

model_name = "csebuetnlp/banglabert"

tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained("config.json")

model = AutoModel.from_pretrained("model.bin",config=config)

classifier = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=3)

# classifier.encoder = model.encoder
# classifier.embeddings = model.embeddings
# model

classifier.electra.embeddings = model.embeddings
classifier.electra.encoder = model.encoder
classifier


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [6]:
# for param in classifier.electra.embeddings.parameters():
#   param.requires_grad=False

for i in range(6):
  for param in classifier.electra.encoder.layer[i].parameters():
    param.requires_grad=False

# for param in classifier.electra.encoder.parameters():
#   param.requires_grad=False

# classifier.electra.encoder.layer[0]

In [ ]:
for name, param in classifier.named_parameters():
  print("name: ",name," ","Trainable: ",param.requires_grad)

name:  electra.embeddings.word_embeddings.weight   Trainable:  False
name:  electra.embeddings.position_embeddings.weight   Trainable:  False
name:  electra.embeddings.token_type_embeddings.weight   Trainable:  False
name:  electra.embeddings.LayerNorm.weight   Trainable:  False
name:  electra.embeddings.LayerNorm.bias   Trainable:  False
name:  electra.encoder.layer.0.attention.self.query.weight   Trainable:  False
name:  electra.encoder.layer.0.attention.self.query.bias   Trainable:  False
name:  electra.encoder.layer.0.attention.self.key.weight   Trainable:  False
name:  electra.encoder.layer.0.attention.self.key.bias   Trainable:  False
name:  electra.encoder.layer.0.attention.self.value.weight   Trainable:  False
name:  electra.encoder.layer.0.attention.self.value.bias   Trainable:  False
name:  electra.encoder.layer.0.attention.output.dense.weight   Trainable:  False
name:  electra.encoder.layer.0.attention.output.dense.bias   Trainable:  False
name:  electra.encoder.layer.0.atte

In [ ]:
# bangla_bert = AutoModelForMaskedLM.from_pretrained("")

# tokenizer_name = 'sagorsarker/bangla-bert-base'

# tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)

train_texts = ["আমার কিছু ভালো লাগে না","আমার সব কিছুই ভালো লাগে"]
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

# train_encodings["input_ids"]

# print(train_encodings['input_ids'])

output = classifier(torch.tensor(train_encodings["input_ids"]))
# print(output[0].shape)
print(output[0].shape)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


torch.Size([2, 3])


In [7]:
import gdown

train_data_link = "https://drive.google.com/uc?id=1mAlatqgef3Jy64ZmHGJ9vNBl3Voxsm21"
gdown.download(train_data_link,"train.csv")

val_data_link = "https://drive.google.com/uc?id=1yWabvxdMa6GWaE_K6cVNI0TylCP3MYHg"
gdown.download(val_data_link,"val.csv")

# fine_tuned_model_path = "https://drive.google.com/uc?id=1VchxbcjhTE4G0AhiJuFbvO_Mnfwhp1Mp"
# gdown.download(fine_tuned_model_path,"fine_tuned_bert.pth")

Downloading...
From: https://drive.google.com/uc?id=1mAlatqgef3Jy64ZmHGJ9vNBl3Voxsm21
To: /content/train.csv
100%|██████████| 1.12M/1.12M [00:00<00:00, 90.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yWabvxdMa6GWaE_K6cVNI0TylCP3MYHg
To: /content/val.csv
100%|██████████| 345k/345k [00:00<00:00, 117MB/s]


'val.csv'

In [8]:
# import pandas as pd
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")

In [ ]:
# train_df.head()

In [9]:
from sklearn.model_selection import train_test_split
def pandaToList(dataset):
    text_list = []
    label_list = []
    for i in range(len(dataset['text'])):
      text_list.append(dataset['text'][i])
      label_list.append(int(dataset['label'][i]))
    return text_list, label_list

train_texts, train_labels =  pandaToList(train_df)
val_texts, val_labels = pandaToList(val_df)

In [10]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# print(train_encodings['input_ids'][0])
# print(val_encodings["input_ids"][0])

In [11]:
import torch

class ViolenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ViolenceDataset(train_encodings, train_labels,)
val_dataset = ViolenceDataset(val_encodings, val_labels)

In [12]:


from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}



In [13]:
from transformers import  Trainer, TrainingArguments, EarlyStoppingCallback, IntervalStrategy, AutoModelForSequenceClassification

training_args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy = IntervalStrategy.STEPS, # "steps"
   #output_dir= output_dir,
  #  output_dir="/content/drive/MyDrive/model_emnlp/buetcsenlp_banglabert_finetuned_classifier",
   eval_steps = 250, # Evaluation and Save happens every 250 steps
   save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.

   learning_rate=2e-5,
   per_device_train_batch_size=1,
   per_device_eval_batch_size=1,
   num_train_epochs=50,
   weight_decay=0.01,
   metric_for_best_model = 'f1',
  #  generator=torch.Generator(device='cuda'),
   load_best_model_at_end=True)

trainer = Trainer(
    model=classifier,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    # generator=torch.Generator(device='cuda'),
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)
# trainer.to('cuda')

In [14]:
trainer.train()

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
250,No log,0.843325,0.650376,0.653245,0.664993,0.638070
500,0.922700,0.734156,0.755639,0.783559,0.693170,0.716771
750,0.922700,1.075887,0.751128,0.724924,0.774510,0.737972
1000,1.083900,1.032765,0.778195,0.754517,0.774739,0.752028
1250,1.083900,0.967025,0.790226,0.760027,0.783737,0.769782
1500,1.049600,1.005005,0.772180,0.741369,0.794404,0.758981
1750,1.049600,0.925057,0.786466,0.749358,0.793305,0.764091
2000,1.064600,0.964165,0.793233,0.758076,0.803946,0.774972
2250,1.064600,1.020650,0.792481,0.756803,0.792489,0.771174
2500,0.960700,1.254827,0.760902,0.731375,0.785962,0.746895


TrainOutput(global_step=5750, training_loss=0.8722142413595448, metrics={'train_runtime': 886.4595, 'train_samples_per_second': 235.036, 'train_steps_per_second': 235.036, 'total_flos': 460962374418000.0, 'train_loss': 0.8722142413595448, 'epoch': 1.38})

In [ ]:
trainer.save_model("/content/drive/MyDrive/model_emnlp/buetcsenlp_banglabert_finetuned_classifier")

In [ ]:
!zip -r /content/drive/MyDrive/model_emnlp/checkpoint_at_9500.zip /content/training_with_callbacks/checkpoint-9500

  adding: content/training_with_callbacks/checkpoint-9500/ (stored 0%)
  adding: content/training_with_callbacks/checkpoint-9500/vocab.txt (deflated 71%)
  adding: content/training_with_callbacks/checkpoint-9500/optimizer.pt (deflated 36%)
  adding: content/training_with_callbacks/checkpoint-9500/special_tokens_map.json (deflated 42%)
  adding: content/training_with_callbacks/checkpoint-9500/scheduler.pt (deflated 49%)
  adding: content/training_with_callbacks/checkpoint-9500/rng_state.pth (deflated 28%)
  adding: content/training_with_callbacks/checkpoint-9500/trainer_state.json (deflated 79%)
  adding: content/training_with_callbacks/checkpoint-9500/tokenizer_config.json (deflated 47%)
  adding: content/training_with_callbacks/checkpoint-9500/tokenizer.json (deflated 76%)
  adding: content/training_with_callbacks/checkpoint-9500/config.json (deflated 54%)
  adding: content/training_with_callbacks/checkpoint-9500/training_args.bin (deflated 48%)
  adding: content/training_with_callbac